# Shareholder Service - Ownership Structure Analysis

Track major shareholders, free float percentages, and ownership distribution for investment analysis.

## Overview

The **Shareholder Service** provides ownership data:

- **Major Shareholders**: Top shareholders with ownership percentages
- **Free Float**: Tradable shares percentage and holder count
- **NVDR Identification**: Non-Voting Depositary Receipt holders
- **Ownership Concentration**: Analyze control and liquidity
- **Foreign vs Local**: Track nationality of major holders

### When to Use

- Liquidity analysis (free float monitoring)
- Control analysis (ownership concentration)
- Corporate governance research
- M&A potential assessment
- Shareholder activism tracking

In [ ]:
!pip install settfex

In [2]:
import asyncio
from settfex.services.set import get_shareholder_data
import pandas as pd

print("Libraries imported!")

Libraries imported!


## Basic Usage

In [3]:
# Fetch shareholder data
data = await get_shareholder_data("MINT")

print(f"Symbol: {data.symbol}")
print(f"Total Shareholders: {data.total_shareholder:,}")
print(f"Book Close Date: {data.book_close_date}")
print(f"\nFree Float:")
print(f"  Percentage: {data.free_float.percent_free_float:.2f}%")
print(f"  Number of Holders: {data.free_float.number_of_holder:,}")
print(f"\nTop 10 Major Shareholders:\n")

for sh in data.major_shareholders[:10]:
    nvdr_flag = " [NVDR]" if sh.is_thai_nvdr else ""
    print(f"{sh.sequence}. {sh.name}{nvdr_flag}")
    print(f"   Nationality: {sh.nationality}")
    print(f"   Shares: {sh.number_of_share:,} ({sh.percent_of_share:.2f}%)")
    print()

Symbol: MINT
Total Shareholders: 48,794
Book Close Date: 2025-09-02 00:00:00+07:00

Free Float:
  Percentage: 59.51%
  Number of Holders: 41,108

Top 10 Major Shareholders:

1. บริษัท  ไมเนอร์ โฮลดิ้ง (ไทย) จำกัด
   Nationality: None
   Shares: 916,556,730 (16.17%)

2. นาย นิติ โอสถานุเคราะห์
   Nationality: None
   Shares: 558,134,428 (9.84%)

3. UBS AG SINGAPORE BRANCH
   Nationality: None
   Shares: 475,893,748 (8.39%)

4. Thai NVDR Company Limited [NVDR]
   Nationality: None
   Shares: 465,082,581 (8.20%)

5. SOUTH EAST ASIA UK (TYPE C) NOMINEES LIMITED
   Nationality: None
   Shares: 232,679,330 (4.10%)

6. BANQUE PICTET & CIE SA
   Nationality: None
   Shares: 202,602,593 (3.57%)

7. นาย วิลเลียม เอ็ลล์วู๊ด ไฮเน็ค
   Nationality: None
   Shares: 188,562,074 (3.33%)

8. SOCIAL SECURITY OFFICE
   Nationality: None
   Shares: 187,751,038 (3.31%)

9. กองทุนรวม  วายุภักษ์หนึ่ง
   Nationality: None
   Shares: 159,527,610 (2.81%)

10. THE BANK OF NEW YORK MELLON
   Nationality: None
   

## Advanced Usage - Ownership Concentration

In [4]:
async def analyze_ownership_concentration(symbols: list[str]):
    """
    Analyze ownership concentration for multiple stocks.
    """
    tasks = [get_shareholder_data(symbol) for symbol in symbols]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    analysis = []
    for symbol, data in zip(symbols, results):
        if isinstance(data, Exception):
            continue
        
        # Calculate concentration metrics
        top5_pct = sum(sh.percent_of_share for sh in data.major_shareholders[:5])
        top10_pct = sum(sh.percent_of_share for sh in data.major_shareholders[:10])
        
        analysis.append({
            "symbol": symbol,
            "total_shareholders": data.total_shareholder,
            "free_float_pct": data.free_float.percent_free_float,
            "top5_ownership": top5_pct,
            "top10_ownership": top10_pct,
            "major_shareholders_count": len(data.major_shareholders)
        })
    
    df = pd.DataFrame(analysis).sort_values('top5_ownership', ascending=False)
    return df

# Analyze blue chips
symbols = ["PTT", "KBANK", "CPALL", "AOT", "BBL", "MINT"]
concentration = await analyze_ownership_concentration(symbols)

print("Ownership Concentration Analysis:\n")
print(concentration.to_string(index=False))

Ownership Concentration Analysis:

symbol  total_shareholders  free_float_pct  top5_ownership  top10_ownership  major_shareholders_count
   AOT              106375       30.000000           77.76            80.40                        10
   PTT              185903       48.880001           70.96            76.86                        10
 CPALL              136961       63.939999           52.26            61.99                        17
  MINT               48794       59.509998           46.70            62.42                        21
   BBL               64388       98.589996           38.70            48.51                        21
 KBANK               80838       82.510002           36.30            48.43                        22


## Use Case: Foreign vs Local Shareholders

In [5]:
# Analyze nationality distribution
data = await get_shareholder_data("CPALL")

thai_holders = [sh for sh in data.major_shareholders if sh.nationality == "THAI"]
foreign_holders = [sh for sh in data.major_shareholders if sh.nationality != "THAI"]

thai_pct = sum(sh.percent_of_share for sh in thai_holders)
foreign_pct = sum(sh.percent_of_share for sh in foreign_holders)

print(f"Ownership by Nationality for {data.symbol}:")
print(f"\nThai Shareholders: {len(thai_holders)} ({thai_pct:.2f}%)")
print(f"Foreign Shareholders: {len(foreign_holders)} ({foreign_pct:.2f}%)")
print(f"\nTop Foreign Shareholders:")
for sh in foreign_holders[:5]:
    print(f"  {sh.name} ({sh.nationality}): {sh.percent_of_share:.2f}%")

Ownership by Nationality for CPALL:

Thai Shareholders: 0 (0.00%)
Foreign Shareholders: 17 (68.14%)

Top Foreign Shareholders:
  บริษัท  ซี.พี.เมอร์แชนไดซิ่ง จำกัด (None): 30.78%
  Thai NVDR Company Limited (None): 12.95%
  SOCIAL SECURITY OFFICE (None): 3.15%
  SOUTH EAST ASIA UK (TYPE C) NOMINEES LIMITED (None): 3.00%
  STATE STREET EUROPE LIMITED (None): 2.38%


## Use Case: NVDR Analysis

In [6]:
# Identify NVDR shareholders
nvdr_holders = [sh for sh in data.major_shareholders if sh.is_thai_nvdr]

print(f"NVDR Shareholders: {len(nvdr_holders)}\n")
for sh in nvdr_holders:
    print(f"{sh.name}")
    print(f"  Ownership: {sh.percent_of_share:.2f}%")
    print(f"  Shares: {sh.number_of_share:,}")
    print()

NVDR Shareholders: 1

Thai NVDR Company Limited
  Ownership: 12.95%
  Shares: 1,163,476,555



## Next Steps

- **[NVDR Holder](07_nvdr_holder.ipynb)** - NVDR-specific data
- **[Company Profile](04_company_profile.ipynb)** - Governance and management
- **[Stock Profile](03_stock_profile.ipynb)** - Foreign ownership limits

**Documentation**: `/docs/settfex/services/set/shareholder.md`